In [1]:
import wrangle
import pandas as pd
import numpy as np
import modeling

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [2]:
train, validate, test = wrangle.wrangle_zillow()

In [3]:
train.shape, validate.shape, test.shape

((27696, 17), (9233, 17), (9233, 17))

In [4]:
train.head().T

,41050,639,9551,17505,49962
parcelid,12447572,12796007,17096073,10829225,12581046
bathrooms,1.0,1.0,2.5,2.0,1.0
bedrooms,3.0,3.0,4.0,3.0,2.0
sqft,1120.0,1513.0,1943.0,1316.0,775.0
county,Los Angeles,Los Angeles,Ventura,Los Angeles,Los Angeles
fireplacecnt,0.0,0.0,1.0,0.0,0.0
garagecarcnt,0.0,0.0,2.0,0.0,0.0
lotsize,5940.0,6324.0,3136.0,6129.0,5885.0
yearbuilt,1947.0,1950.0,2012.0,1949.0,1943.0
poolcnt,0.0,0.0,0.0,0.0,0.0


In [12]:
# scale features
train.columns

Index(['parcelid', 'bathrooms', 'bedrooms', 'sqft', 'county', 'fireplacecnt',
       'garagecarcnt', 'lotsize', 'yearbuilt', 'poolcnt', 'logerror', 'tract',
       'tax_value', 'Orange', 'Ventura', '4plusBath', '3to5garage'],
      dtype='object')

In [5]:
features = ['bedrooms', 'sqft', 'fireplacecnt', 'garagecarcnt', 'lotsize', 'yearbuilt', 'poolcnt','4plusBath',
            'Orange', 'Ventura']

In [6]:
X_train = train[features]
y_train = train['tax_value']

X_validate = validate[features]
y_validate = validate['tax_value']

X_test = test[features]
y_test = test['tax_value']

In [7]:
scaler = QuantileTransformer(output_distribution = 'normal')
s_train = modeling.scaler(scaler, X_train, y_train)

In [8]:
s_train

array([[-0.23550885, -1.01877814, -5.19933758, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [-0.23550885, -0.16128127, -5.19933758, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [ 0.78162595,  0.43210422,  1.40453311, ..., -5.19933758,
        -5.19933758,  5.19933758],
       ...,
       [-0.23550885, -1.19976601, -5.19933758, ..., -5.19933758,
        -5.19933758, -5.19933758],
       [-0.23550885,  0.29660674,  1.40453311, ..., -5.19933758,
         5.19933758, -5.19933758],
       [-0.23550885, -0.16001041, -5.19933758, ..., -5.19933758,
        -5.19933758, -5.19933758]])

In [10]:
list(reversed(range(1,11)))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]